
![alt text](https://i.ibb.co/JKd0WF5/SF-Post-2.jpg)

This Colab notebook allows you to compare two Screaming Frog crawls and outputs summary tables and charts of all the differences.

Credit due to Alessio Nittoli [(@Nittolese)](https://https://twitter.com/nittolese) for creating the initial script. 

Besides these summary tables and charts, I’ve taken the script further by adding the following functionalities:

*   You can now select the type of content you want to compare by:
   - discarding redirects
   - discarding non-HTML elements (application, PDFs, images etc.) to only focus on HTML URLs

*   More [Colab forms](https://colab.research.google.com/notebooks/forms.ipynb) for easier parameterisation.
*   Additional guidance for newcomers.

More info in [my blog post](https://www.charlywargnier.com/python-script-to-compare-screaming-frog-crawls/).


# Part I: Setting things up!

## Export your crawl files from Screaming Frog

First thing first, you need to export your crawl data from Screaming Frog. We want `internall all` CSVs, which contain all per-url information.

Once crawl #01 and #02 are completed go to 'internal' tab and click on the ‘export’ button located in the top-left corner:



![alt text](https://i.ibb.co/VMxR8m3/2020-03-17-21-52-36.jpg)




## Upload your files to Google Drive & connect Drive to Colab

Head to https://www.google.com/drive/ and drop both CSV exports in your preferred folder. 

Once uploaded, connect to Google Drive to Colab with this snippet of code:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Make the crawl folder the root path in Colab


Now that Drive and Colab are linked, navigate to the folder where you uploaded your crawls via Colab's navigator:

![alt text](https://i.ibb.co/yhcpp9V/2020-03-19-16-31-06.jpg)


Then, right-click on the folder path to copy it and paste it into the Python `root_path` variable in the code cell below:

In [ ]:
root_path = '/content/drive/My Drive/Data_Science/Colab_Notebooks/Top_Python_Projects/SFrog_Compare_Tool/CSVs/'
print("root_path is now: '" + root_path + "'")

root_path is now: '/content/drive/My Drive/Data_Science/Colab_Notebooks/Top_Python_Projects/SFrog_Compare_Tool/CSVs/'


## Install Pygsheets, Numpy, Pandas and Re

We now need to install `pygsheets` , a Python library we'll use to interact with our Google spreadsheet, through the Google Sheets API.

More info about here: https://pygsheets.readthedocs.io/en/stable/



In [ ]:
!pip install --upgrade -q pygsheets

Numpy, Pandas and Re are useful libraries for data manipulation. Let's import them via the code snippet below: 

In [ ]:
import numpy as np
import pandas as pd
import re #For regex manipulations

Then, we need to connect Colab to Google Sheets. We need an authentication key to access the Google Spreadsheet API:

In [ ]:
import google.auth
from google.colab import auth

auth.authenticate_user()
import pygsheets
credentials, _ = google.auth.default()
gc = pygsheets.client.Client(credentials)

## Create a Google sheet to host your comparison data

Let's create a Google sheet and give it a name via the form below:

In [ ]:
sh_name = 'file_name' #@param {type:"string"}
sh = gc.create(sh_name)

Note that this new sheet will be located in the root folder of your Google Drive

## Convert your CSVs to Pandas Dataframes

Add the names of your CSV files in the form fields below (no need to add the .csv extension).

These files are located in your crawl folder. You can check all the files located in your crawl folder via the Unix command `ls`:

In [ ]:
!ls -lah drive/My\ Drive/Data_Science/Colab_Notebooks/Top_Python_Projects/SFrog_Compare_Tool/CSVs

total 8.3M
drwx------ 2 root root 4.0K Dec  4 18:29  Archives
-rw------- 1 root root 3.6M Mar 21 20:15  internal_all_new.csv
-rw------- 1 root root 4.7M Mar 21 20:15  internal_all_previous.csv
drwx------ 2 root root 4.0K Mar 20 10:39  Misc
drwx------ 2 root root 4.0K Mar 21 20:08 'Valid yet non retained crawls'
drwx------ 2 root root 4.0K Mar 21 20:14 'VHOLS crawls'


In [ ]:
Previous_CSV = 'internal_all_new' #@param {type:"string"}
Current_CSV = 'internal_all_previous' #@param {type:"string"}

The code cell below will convert both CSVs to Pandas dataframes, called `df_prev` and `df_current` respectively.



In [ ]:
df_prev = pd.read_csv(root_path+Previous_CSV+'.csv',header=1, low_memory=True)
df_current = pd.read_csv(root_path+Current_CSV+'.csv',header=1, low_memory=True)

## Select which type of content you want to compare

Via the form below you can discard redirects and/or non HTML elements (application, PDFs, images etc.) from the comparison:




In [ ]:
#Classifier for df_prev
df_prev['Content'].fillna('Misc', inplace=True)
df_prev = df_prev.rename(columns={"Status Code": "StatusCode"})
df_prev['StatusCode'] = df_prev['StatusCode'].apply(str)

df_prev['ContentClass'] = pd.np.where(df_prev.Content.str.contains("html"), 'HTML URLs',
      pd.np.where(df_prev.StatusCode.str.contains("^3.*"), 'Redirect','Other'))

#Classifier for df_current
df_current['Content'].fillna('Misc', inplace=True)
df_current = df_current.rename(columns={"Status Code": "StatusCode"})
df_current['StatusCode'] = df_current['StatusCode'].apply(str)

df_current['ContentClass'] = pd.np.where(df_current.Content.str.contains("html"), 'HTML URLs',
      pd.np.where(df_current.StatusCode.str.contains("^3.*"), 'Redirect','Other'))

myNewList = []

Remove_redirects = False  # @param {type:"boolean"}
Remove_non_html = False  # @param {type:"boolean"}

if Remove_redirects == True:
    myNewList.append('Redirect')
if Remove_non_html == True:
    myNewList.append('Other')

print (myNewList)

current = df_current[~df_current['ContentClass'].isin(myNewList)]
previous = df_prev[~df_prev['ContentClass'].isin(myNewList)]


[]


Now that we're all set, let's start gathering crawl changes!

# Part II: Gathering crawl changes

## New found pages



Via Python's function `pd.merge` we are going to merge the previous and current reports using as their common key the URLs.

For more about this [check the docs](https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html) .

In [ ]:
pre_current_master = pd.merge(previous,\
                            current,\
                            suffixes=('_prev', '_current'),\
                            on='Address',\
                            how='outer')

We'll filter out all null values.

In [ ]:
_new_found_pages = pre_current_master[pre_current_master['StatusCode_prev'].isna()]

And keep only the columns that contains `_current` (from the latest crawl) and `Address`. 

In [ ]:
new_found_pages = _new_found_pages.filter(regex='Address|\_current')

Now we're going to evaluate if the `new_found_pages` DataFrame is empty. If not, we are going to create a new worksheet and append the DataFrame.

In [ ]:
if len(new_found_pages['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now we need to select the first Sheet in our Spreadsheet. 
  #To access it, we need to select it through an index (0-based).
  wks = sh[0]
  #set_dataframe is a method that injects a DataFrame into a spreadsheet. 
  #The first argument is the DF and the second arg is the cell where we start filling.
  wks.set_dataframe(new_found_pages, 'A1',fit=True)
  #We can also change the title of our worksheet, in this way
  sh.sheet1.title="New Found Pages"

## Newly lost pages

In [ ]:
_new_lost_pages = pre_current_master[pre_current_master['StatusCode_current'].isna()]

In [ ]:
new_lost_pages = _new_lost_pages.filter(regex='Address|\_prev')

In [ ]:
if len(new_lost_pages['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('New Lost Pages')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(new_lost_pages, 'A1',fit=True)

## Changed status codes

In [ ]:
_changed_status_code = pd.merge(df_prev[['Address','StatusCode']],\
                                df_current[['Address','StatusCode']],\
                                suffixes=('_prev', '_current'),\
                                on='Address',\
                                how='inner')

_changed_status_code['diff'] = np.where(_changed_status_code['StatusCode_prev'] == _changed_status_code['StatusCode_current'], 'no change', 'changed')
_changed_status_code

changed_status_code = _changed_status_code[_changed_status_code['diff'] == 'changed']

In [ ]:
if len(changed_status_code['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Status Code')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_status_code, 'A1',fit=True)

## Changed indexation 

In [ ]:
_changed_indexation = pd.merge(previous[['Address','StatusCode', 'Indexability','Indexability Status']],\
                                current[['Address','StatusCode', 'Indexability','Indexability Status']],\
                                suffixes=('_prev', '_current'), on='Address', how='inner')

In [ ]:
_changed_indexation['diff'] = np.where(_changed_indexation['Indexability_prev'] == _changed_indexation['Indexability_current'], 'no change', 'changed')

In [ ]:
#Dataframe containing 'changed' AND 'unchanged'
changed_indexation_all = _changed_indexation[['Address','Indexability_prev','Indexability Status_prev','Indexability_current','Indexability Status_current','diff']]

In [ ]:
##Dataframe containing only 'changed' elements
changed_indexation = changed_indexation_all[changed_indexation_all['diff'] == 'changed']

In [ ]:
if len(changed_indexation['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Indexation')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_indexation, 'A1',fit=True)

## Changed canonicals



In [ ]:
#*_all* dataframes are used for the Crawl Master tab.
#non *_all* dataframes are used for changes tab (individual tabs). Drop.na removes removes NaN, thus removes 'unchanged' elements

changed_cano_all = pd.merge(previous[['Address','StatusCode','Canonical Link Element 1']],current[['Address','StatusCode','Canonical Link Element 1']],\
                               suffixes=('_prev', '_current'), on='Address', how='inner')

changed_cano_all.fillna("No value", inplace = True)
changed_cano_all['diff_cano'] = np.where(changed_cano_all['Canonical Link Element 1_prev'] == changed_cano_all['Canonical Link Element 1_current'], 'no change', 'changed')
changed_cano_all = changed_cano_all[~changed_cano_all['Canonical Link Element 1_current'].str.contains('No value', na=False)]

In [ ]:
_changed_canonical = pd.merge(previous[['Address','StatusCode','Canonical Link Element 1']],current[['Address','StatusCode','Canonical Link Element 1']],\
                               suffixes=('_prev', '_current'), on='Address', how='inner')

In [ ]:
_changed_canonical.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
_changed_canonical['diff_canonical'] = np.where(_changed_canonical['Canonical Link Element 1_prev'] == _changed_canonical['Canonical Link Element 1_current'], 'no change', 'changed')

In [ ]:
changed_canonical = _changed_canonical[_changed_canonical['diff_canonical'] == 'changed']

In [ ]:
if len(changed_canonical['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Canonicals')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_canonical, 'A1',fit=True)

## Changed meta titles

In [ ]:
_changed_meta = pd.merge(previous[['Address','Title 1', 'Meta Description 1']],\
                                current[['Address','Title 1', 'Meta Description 1']],\
                                suffixes=('_prev', '_current'), on='Address', how='inner')

In [ ]:
_changed_meta.fillna("No value", inplace = True)
_changed_meta['diff_title'] = np.where(_changed_meta['Title 1_prev'] == _changed_meta['Title 1_current'], 'no change', 'changed')
changed_title_all = _changed_meta.filter(regex='Address|^Title.+|diff\_title')
changed_title_all = changed_title_all[~changed_title_all['Title 1_current'].str.contains('No value', na=False)]

In [ ]:
changed_title = _changed_meta[_changed_meta['diff_title'] == 'changed'].dropna().filter(regex='Address|^Title.+|diff\_title')

In [ ]:
if len(changed_title['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Title')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_title, 'A1',fit=True)

## Changed meta descriptions


In [ ]:
_changed_meta['diff_desc'] = np.where(_changed_meta['Meta Description 1_prev'] == _changed_meta['Meta Description 1_current'], 'no change', 'changed')

In [ ]:
changed_desc_all = _changed_meta.filter(regex='Address|.+Description.+|diff\_desc')
changed_desc_all = changed_desc_all[~changed_desc_all['Meta Description 1_current'].str.contains('No value', na=False)]

In [ ]:
changed_desc = _changed_meta[_changed_meta['diff_desc'] == 'changed'].dropna().filter(regex='Address|.+Description.+|diff\_desc')

In [ ]:
if len(changed_desc['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed Description')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_desc, 'A1',fit=True)

## Changed H1 tags

In [ ]:
changed_h1_all = pd.merge(previous.filter(regex='Address|^H1\-\d{1,2}$'),\
                            current.filter(regex='Address|^H1\-\d{1,2}$'),\
                            suffixes=('_prev', '_current'),\
                            on='Address',\
                            how='inner')

changed_h1_all.fillna("No value", inplace = True)
changed_h1_all = changed_h1_all[~changed_h1_all['H1-1_current'].str.contains('No value', na=False)]
changed_h1_all['diff_h1'] = np.where(changed_h1_all['H1-1_prev'] == changed_h1_all['H1-1_current'], 'no change', 'changed')

In [ ]:
changed_h1 = pd.merge(previous.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3),\
                            current.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3),\
                            suffixes=('_prev', '_current'),\
                            on='Address',\
                            how='inner')

In [ ]:
changed_h1.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
changed_h1_cols = current.filter(regex='Address|^H1\-\d{1,2}$').dropna(thresh=3).columns.tolist()

In [ ]:
r = re.compile("^H1\-\d{1,2}")

In [ ]:
newlist = list(filter(r.match, changed_h1_cols))

In [ ]:
for i in range(1,len(newlist)+1):
    new_diff = "H1-"+str(i)
    new = f"{new_diff}_current"
    old = f"{new_diff}_prev"
    _tmp = f"diff-{new_diff}"
    changed_h1[_tmp] = changed_h1[new] != changed_h1[old]

In [ ]:
changed_h1.columns =[column.replace("-", "_") for column in changed_h1.columns] 

In [ ]:
diff_col_lst = changed_h1.filter(regex='diff\_H1').columns.tolist()

In [ ]:
exp = ' or '.join(diff_col_lst)

In [ ]:
changed_h1 = changed_h1.query(exp)

In [ ]:
if len(changed_h1['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed H1')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_h1, 'A1',fit=True)

## Changed H2 tags

In [ ]:
changed_h2_1_all = pd.merge(previous.filter(regex='Address|^H2\-\d{1,2}$'),\
                            current.filter(regex='Address|^H2\-\d{1,2}$'),\
                            suffixes=('_prev', '_current'),\
                            on='Address',\
                            how='inner')

changed_h2_1_all.fillna("No value", inplace = True)
changed_h2_1_all = changed_h2_1_all[~changed_h2_1_all['H2-1_current'].str.contains('No value', na=False)]
changed_h2_1_all['diff_h2_1'] = np.where(changed_h2_1_all['H2-1_prev'] == changed_h2_1_all['H2-1_current'], 'no change', 'changed')

In [ ]:
changed_h2_2_all = pd.merge(previous.filter(regex='Address|^H2\-\d{1,2}$'),\
                            current.filter(regex='Address|^H2\-\d{1,2}$'),\
                            suffixes=('_prev', '_current'),\
                            on='Address',\
                            how='inner')

changed_h2_2_all.fillna("No value", inplace = True)

changed_h2_2_all = changed_h2_2_all[~changed_h2_2_all['H2-2_current'].str.contains('No value', na=False)]
changed_h2_2_all['diff_h2_2'] = np.where(changed_h2_2_all['H2-2_prev'] == changed_h2_2_all['H2-2_current'], 'no change', 'changed')

In [ ]:
changed_h2 = pd.merge(previous.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3),\
                current.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3),\
                suffixes=('_prev', '_current'),\
                on='Address',\
                how='inner')

In [ ]:
changed_h2.replace(np.nan, '', regex=True, inplace=True)

In [ ]:
changed_h2_cols = current.filter(regex='Address|^H2\-\d{1,2}$').dropna(thresh=3).columns.tolist()

In [ ]:
r = re.compile("^H2\-\d{1,2}")

In [ ]:
newlist = list(filter(r.match, changed_h2_cols))

In [ ]:
for i in range(1,len(newlist)+1):
    new_diff = "H2-"+str(i)
    new = f"{new_diff}_current"
    old = f"{new_diff}_prev"
    _tmp = f"diff-{new_diff}"
    changed_h2[_tmp] = changed_h2[new] != changed_h2[old]

In [ ]:
changed_h2.columns =[column.replace("-", "_") for column in changed_h2.columns] 

In [ ]:
diff_col_lst = changed_h2.filter(regex='diff\_H2').columns.tolist()

In [ ]:
exp = ' or '.join(diff_col_lst)

In [ ]:
changed_h2 = changed_h2.query(exp)

In [ ]:
if len(changed_h2['Address'].tolist()) == 0:
  print('No changes detected!')
else:
  #Now let's add a new worksheet with a title
  wks = sh.add_worksheet('Changed H2')
  #Append a DataFrame in our new worksheet
  wks.set_dataframe(changed_h2, 'A1',fit=True)

# Part III: Aggregating all crawl changes in a Master sheet

In this section, we'll create a Master sheet called `Crawl Master` (located in tab 01), containing aggregated tables & charts for each tab/report.



##Status codes overview

Create an aggregated table for 'previous' status codes count.


In [ ]:
#Create a pivot table for previous status codes
pivot_table_pre = pd.pivot_table(_changed_status_code, index='StatusCode_prev', aggfunc='count')
#Rename Index
pivot_table_pre = pivot_table_pre.rename_axis(None, axis=1).rename_axis('Status Code', axis=0)
#Delete/Rename columns  
del pivot_table_pre['diff']
del pivot_table_pre['StatusCode_current']
pivot_table_pre.rename(columns={'Address':'Count'}, inplace=True)

Create an aggregated table for 'current' status codes count.


In [ ]:
#Create a pivot table for current status codes
pivot_table_current = pd.pivot_table(_changed_status_code, index='StatusCode_current', aggfunc='count')
#Rename Index
pivot_table_current = pivot_table_current.rename_axis(None, axis=1).rename_axis('Status Code', axis=0)
#Delete/Rename columns  
del pivot_table_current['diff']
del pivot_table_current['StatusCode_prev']
pivot_table_current.rename(columns={'Address':'Count'}, inplace=True)

Merge both aggregated/pivot tables 'previous' and 'current'

In [ ]:
#Merge both pre and current tables
all_codes = pd.merge(pivot_table_pre,\
                            pivot_table_current,\
                            suffixes=('_prev', '_current'),\
                            on='Status Code',\
                            how='outer')
#Replace N/A by '0'
all_codes.fillna(0, inplace=True)
#Create a 'diff' column to highlight previous vs current status codes count
all_codes['Diff'] = all_codes['Count_current'] - all_codes['Count_prev']
#Add numeric index
all_codes = all_codes.reset_index()

Paste the output into the new 'Overviews' tab 

In [ ]:
#Now let's add a new worksheet with a title
wks = sh.add_worksheet('Crawl Master')
#Append a DataFrame in our new worksheet
wks.set_dataframe(all_codes, 'A1',fit=False)

Create a 'Status Codes Overview' chart

In [ ]:
#Create a chart for status codes
wks.add_chart(('A1', 'A10'), [('B1', 'B10'), ('C1', 'C10')], 'Status Codes Overview')

<Chart COLUMN 'Status Codes Overview'>

## Indexation overview

Create an aggregated table for pre/post indexation count

In [ ]:
#Create a pivot table
Index_Overview = pd.pivot_table(changed_indexation_all, index='diff', aggfunc='count')
#Add numeric index
Index_Overview = Index_Overview.reset_index()
#Remove unwanted columns
Index_Overview = Index_Overview[['Address']]
#Rename column
Index_Overview.rename(columns={'Address':'# Indexable URLs changed'}, inplace=True)
#Now let's add a new worksheet with a title
wks = sh.add_worksheet('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(Index_Overview, 'A1',fit=False)

##Canonical overview

Create an aggregated table for pre/post indexation count

In [ ]:
#Create a pivot table
Cano_Overview = pd.pivot_table(changed_cano_all, index='diff_cano', aggfunc='count')
#Add numeric index
Cano_Overview = Cano_Overview.reset_index()
#Remove unwanted columns
Cano_Overview = Cano_Overview[['Canonical Link Element 1_current']]
'''try:
    Cano_Overview = Cano_Overview[['Canonical Link Element 1_current']]
except KeyError:
    print("no canonical data in the data set")'''
#Rename column
Cano_Overview.rename(columns={'Canonical Link Element 1_current':'# Canos changed'}, inplace=True)
#Select the 'Hidden placeholder' worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(Cano_Overview, 'B1',fit=False)

##Meta title overview

Create an aggregated table for title tags count

In [ ]:
#Create a pivot table
TitlesOverview = pd.pivot_table(changed_title_all, index='diff_title', aggfunc='count')
#Add numeric index
TitlesOverview = TitlesOverview.reset_index()
try:
    #Remove unwanted columns
    TitlesOverview = TitlesOverview[['Title 1_current']]
except KeyError:
    print("no title data in the data set")
#Rename column
TitlesOverview.rename(columns={'Title 1_current':'# Titles changed'}, inplace=True)
# Select the Overview worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(TitlesOverview,'C1',fit=False)

##Meta description overview

Create an aggregated table for pre/post indexation count

In [ ]:
#Create a pivot table
DescrOverview = pd.pivot_table(changed_desc_all, index='diff_desc', aggfunc='count')
#Add numeric index
DescrOverview = DescrOverview.reset_index()
try:
    #Remove unwanted columns
    DescrOverview = DescrOverview[['Meta Description 1_current']]
except KeyError:
    print("no description data in the data set")
#Rename column
DescrOverview.rename(columns={'Meta Description 1_current':'# Descr. changed'}, inplace=True)
#Select the 'Hidden placeholder' worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(DescrOverview,'D1',fit=False)

##H1 overview

Create an aggregated table for pre/post H1 count

In [ ]:
#Create a pivot table
H1_Overview = pd.pivot_table(changed_h1_all, index='diff_h1', aggfunc='count')
#Add numeric index
H1_Overview = H1_Overview.reset_index()
try:
    #Remove unwanted columns
    H1_Overview = H1_Overview[['H1-1_current']]
except KeyError:
    print("no H1 data in the data set")
#Rename column
H1_Overview.rename(columns={'H1-1_current':'# H1s changed'}, inplace=True)
# Select the Overview worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(H1_Overview, 'E1',fit=False)

##H2.1 overview

In [ ]:
#Create a pivot table
H2_1_Overview = pd.pivot_table(changed_h2_1_all, index='diff_h2_1', aggfunc='count')
#Add numeric index
H2_1_Overview = H2_1_Overview.reset_index()
try:
    #Remove unwanted columns
    H2_1_Overview = H2_1_Overview[['H2-1_current']]
except KeyError:
    print("no H2_1 data in the data set")
#Rename column
H2_1_Overview.rename(columns={'H2-1_current':'# H2s(1) changed'}, inplace=True)
# Select the Overview worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(H2_1_Overview, 'F1',fit=False)

##H2.2 overview

In [ ]:
#Create a pivot table
H2_2_Overview = pd.pivot_table(changed_h2_2_all, index='diff_h2_2', aggfunc='count')
#Add numeric index
H2_2_Overview = H2_2_Overview.reset_index()
try:
    #Remove unwanted columns
    H2_2_Overview = H2_2_Overview[['H2-2_current']]
except KeyError:
    print("no H2_2 data in the data set")
#Rename column
H2_2_Overview.rename(columns={'H2-2_current':'# H2s(2) changed'}, inplace=True)
# Select the Overview worksheet
wks = sh.worksheet_by_title('Hidden placeholder')
#Append the DataFrame in the Overview worksheet
wks.set_dataframe(H2_2_Overview, 'G1',fit=False)

# Part IV: Formatting tables & plotting charts  

## Mark-ups table formating + plotting 

In [ ]:
wks = sh.worksheet_by_title('Hidden placeholder')
#Fetch the data from Google sheets, then re-convert into a dataframe we will call df_new
values_mat = wks.get_values(start=(1,1), end=(20,20), returnas='matrix')
cell_matrix = wks.get_all_values(returnas='matrix')
#create df_new
df_new = pd.DataFrame.from_records(values_mat)
#Make the 1st row a header
df_new.columns = df_new.iloc[0]
df_new = df_new[1:]
#remove unwanted columns (only retains columns containing the word 'changed')
df_new = df_new.filter(regex='changed')
#transpose table
df_new = df_new.transpose()
#Reset index
df_new = df_new.reset_index()
#Add Column names
df_new.rename(columns={ df_new.columns[0]: "SEO Mark-up" }, inplace = True)
df_new.rename(columns={ df_new.columns[1]: "Changed" }, inplace = True)
df_new.rename(columns={ df_new.columns[2]: "Unchanged" }, inplace = True)

'''try:
    #Remove unwanted columns
    df_new.rename(columns={ df_new.columns[2]: "Unchanged" }, inplace = True)
except IndexError:
    print("")'''

#Hide the placeholder tab
wks.hidden = True
#Select the 'Crawl Master' tab
wks = sh.worksheet_by_title('Crawl Master')
#Create the chart for the 2nd table
wks.set_dataframe(df_new, 'H1',fit=False)
wks.add_chart(('H1', 'H10'), [('J1', 'J10'), ('I1', 'I10')], 'Mark-up Changes Overview')
#Make the 'Crawl Master' tab the 1st tab in the workbook
wks.index = 0 # index start at 1 , not 0
wks = sh.worksheet_by_title('Crawl Master')
#Insert rows
wks.insert_rows(row=0, number=7)
#Adjust_column_width
wks.adjust_column_width(7, 8, pixel_size=200)

##Adding titles & subtitles


Add the main title


In [ ]:
c1 = wks.cell('A2')
#Set text value
c1.value = 'Crawl Master'
#Set font size
c1.set_text_format('fontSize', 20)
#Set font style = bold
c1.text_format['bold'] = True
#Align title 'left' 
c1.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c1.update()

Add the main subtitle

In [ ]:
c2 = wks.cell('A3')
#Set text value
c2.value = 'Comparison between previous crawl and current crawl'
#Set font size
c2.set_text_format('fontSize', 12)
#Align subtitle 'left' 
c2.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c2.update()

Add a title to the left-hand table

In [ ]:
c3 = wks.cell('A6')
#Set text value
c3.value = 'Status Codes Overview'
#Set font size
c3.set_text_format('fontSize', 13)
#Set font style = bold
c3.text_format['bold'] = True
#Align title 'left' 
c3.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c3.update()

Add a subtitle to the left-hand table

In [ ]:
##Add a subtitle
c3sub = wks.cell('A7')
#Set text value
c3sub.value = 'This overview spans all URLs (incl. non html and redirects)'
#Set font size
c3sub.set_text_format('fontSize', 10)
#Set font style = bold
c3sub.text_format['italic'] = True
#Align title 'left' 
c3sub.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c3sub.update()

Add 'Diff (in %)' heading


In [ ]:
e8 = wks.cell('E8')
#Set text value
e8.value = 'Diff (in %)'

Add a title to the right-hand table

In [ ]:
#Add a title
c4 = wks.cell('H6')
#Set text value
c4.value = 'Mark-up Changes Overview'
#Set font size
c4.set_text_format('fontSize', 13)
#Set font style = bold
c4.text_format['bold'] = True
#Align title 'left' 
c4.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c4.update()

Add a subtitle to the right-hand table

In [ ]:
##Add a subtitle
c4sub = wks.cell('H7')
#Set text value
c4sub.value = 'Mark-ups are only compared when URLs are available in both crawls'
#Set font size
c4sub.set_text_format('fontSize', 10)
#Set font style = bold
c4sub.text_format['italic'] = True
#Align title 'left' 
c4sub.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.LEFT
#Update cell
c4sub.update()

Add a 'Total' heading

In [ ]:
## Add 'Total' heading
k8 = wks.cell('K8')
#Set text value
k8.value = 'Total'

Add '%age change' heading


In [ ]:
l8 = wks.cell('L8')
#Set text value
l8.value = '%age change*'

Right-hand table's 'Nota Bene'


In [ ]:
c4NB = wks.cell('L7')
#Set text value
c4NB.value = '*vs total (K)'
#Set font size
c4NB.set_text_format('fontSize', 10)
#Set font style = bold
c4NB.text_format['italic'] = True
#Align title 'left' 
c4NB.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.CENTER
#Update cell
c4NB.update()

## Add sum and percentage formulas 


Create Sum formulas


In [ ]:
#Get a range of cells
ck9 = wks.cell('K9')
ck10 = wks.cell('K10')
ck11 = wks.cell('K11')
ck12 = wks.cell('K12')
ck13 = wks.cell('K13')
ck14 = wks.cell('K14')
ck15 = wks.cell('K15')

#Apply sum formulas to the range of cells above
ck9.formula = 'I9+J9'
ck10.formula = 'I10+J10'
ck11.formula = 'I11+J11'
ck12.formula = 'I12+J12'
ck13.formula = 'I13+J13'
ck14.formula = 'I14+J14'
ck15.formula = 'I15+J15'

Create Percentage formulas for right-hand table


In [ ]:
#Get the range of cells
cl9 = wks.cell('L9')
cl10 = wks.cell('L10')
cl11 = wks.cell('L11')
cl12 = wks.cell('L12')
cl13 = wks.cell('L13')
cl14 = wks.cell('L14')
cl15 = wks.cell('L15')

#Apply %age formulas to the range of cells
cl9.formula = 'I9/K9'
cl10.formula = 'I10/K10'
cl11.formula = 'I11/K11'
cl12.formula = 'I12/K12'
cl13.formula = 'I13/K13'
cl14.formula = 'I14/K14'
cl15.formula = 'I15/K15'

Create Percentage formulas to left hand table

In [ ]:
#Get the range of cells
ce9 = wks.cell('E9')
ce10 = wks.cell('E10')
ce11 = wks.cell('E11')
ce12 = wks.cell('E12')
ce13 = wks.cell('E13')
ce14 = wks.cell('E14')
ce15 = wks.cell('E15')
ce16 = wks.cell('E16')
ce17 = wks.cell('E17')

#Apply %age formulas to the range of cells
ce9.formula = 'IF(B9="","",IFERROR(((C9-B9)/B9),"n/a"))'
ce10.formula = 'IF(B10="","",IFERROR(((C10-B10)/B10),"n/a"))'
ce11.formula = 'IF(B11="","",IFERROR(((C11-B11)/B11),"n/a"))'
ce12.formula = 'IF(B12="","",IFERROR(((C12-B12)/B12),"n/a"))'
ce13.formula = 'IF(B13="","",IFERROR(((C13-B13)/B13),"n/a"))'
ce14.formula = 'IF(B14="","",IFERROR(((C14-B14)/B14),"n/a"))'
ce15.formula = 'IF(B15="","",IFERROR(((C15-B15)/B15),"n/a"))'
ce16.formula = 'IF(B16="","",IFERROR(((C16-B16)/B16),"n/a"))'
ce17.formula = 'IF(B17="","",IFERROR(((C17-B17)/B17),"n/a"))'

## Final formatting bits 


Percentage formatting

In [ ]:

model_cell4 = wks.cell('N1')
model_cell4.set_number_format(pygsheets.FormatType.NUMBER, '0.0%')
model_cell4.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.CENTER
wks.range('E9:E17', returnas='range').apply_format(model_cell4)
wks.range('L9:L17', returnas='range').apply_format(model_cell4)

Tables' headers formatting

In [ ]:
model_cell = wks.cell('A8')
model_cell.set_text_format('bold', True)
model_cell.color = (0.88,0.88,0.88)
model_cell.horizontal_alignment = pygsheets.custom_types.HorizontalAlignment.CENTER
wks.range('B8:E8', returnas='range').apply_format(model_cell)
wks.range('H8:L8', returnas='range').apply_format(model_cell)

Numbers formatting (add commas)


In [ ]:
model_cell2 = wks.cell('B10')
model_cell2.set_number_format(pygsheets.FormatType.NUMBER, '#,##0_);(#,##0)')
wks.range('B9:C18', returnas='range').apply_format(model_cell2)
wks.range('I9:J18', returnas='range').apply_format(model_cell2)